In [197]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [198]:
con = sqlite3.connect("../data/checking-logs.sqlite.sqlite copy")

## get the schema of the table test

In [199]:
pd.io.sql.read_sql("PRAGMA table_info(test);", con)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [200]:
pd.io.sql.read_sql("SELECT * FROM test LIMIT 10", con)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

In [201]:
pd.io.sql.read_sql("SELECT * FROM deadlines LIMIT 10", con)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [202]:
query = "SELECT t.uid, MIN((strftime('%s', t.first_commit_ts)) - d.deadlines) / 3600 AS 'min_diff' " \
        "FROM test t LEFT JOIN deadlines d ON d.labs=t.labname WHERE t.labname IS NOT 'project1'"
df_min = pd.io.sql.read_sql(query, con, parse_dates=['first_commit_ts', 'first_view_ts'], index_col='uid')
df_min

,min_diff
uid,
user_30,-202


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [203]:
query = "SELECT t.uid, MAX((strftime('%s', t.first_commit_ts)) - d.deadlines) / 3600 AS 'max_diff' " \
        "FROM test t LEFT JOIN deadlines d ON d.labs=t.labname WHERE t.labname IS NOT 'project1'"
df_max = pd.io.sql.read_sql(query, con, parse_dates=['first_commit_ts', 'first_view_ts'], index_col='uid')
df_max

,max_diff
uid,
user_25,-2


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [204]:
query = "SELECT t.uid, AVG((strftime('%s', t.first_commit_ts)) - d.deadlines) / 3600 AS 'avg_diff' " \
        "FROM test t LEFT JOIN deadlines d ON d.labs=t.labname WHERE t.labname IS NOT 'project1'"
df_avg = pd.io.sql.read_sql(query, con, parse_dates=['first_commit_ts', 'first_view_ts'], index_col='uid')
df_avg

,avg_diff
uid,
user_1,-89.687841


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [205]:
query = "SELECT AVG(diff), AVG((strftime('%s', t.first_commit_ts)) - d.deadlines) / 3600 AS 'avg_diff' " \
        "FROM test t LEFT JOIN deadlines d ON d.labs=t.labname WHERE t.labname IS NOT 'project1'"
views_diff = pd.io.sql.read_sql(query, con, parse_dates=['first_commit_ts', 'first_view_ts'], index_col='uid')
views_diff

DatabaseError: Execution failed on sql 'SELECT AVG(diff), AVG((strftime('%s', t.first_commit_ts)) - d.deadlines) / 3600 AS 'avg_diff' FROM test t LEFT JOIN deadlines d ON d.labs=t.labname WHERE t.labname IS NOT 'project1'': no such column: diff

## close the connection

In [ ]:
con.close()